In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import os
import cv2

#To run the prediction on the GPU, else comment out this line to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from csbdeep.utils.tf import limit_gpu_memory
from skimage.filters import threshold_otsu
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.path.append('../../Terminator/')

from IPython.display import clear_output
import glob
from tifffile import imread
import tqdm
from csbdeep.utils import Path, normalize
from stardist import random_label_cmap, _draw_polygons,dist_to_coord
from stardist.models import StarDist2D
from TerminatorUtils.helpers import normalizeFloat, OtsuThreshold2D, save_8bit_tiff_imagej_compatible, save_16bit_tiff_imagej_compatible,Integer_to_border,SeedStarDistWatershed
from TerminatorUtils.helpers import Prob_to_Binary, zero_pad
from TerminatorUtils.helpers import save_tiff_imagej_compatible
from csbdeep.models import Config, CARE
np.random.seed(6)
lbl_cmap = random_label_cmap()

from skimage.morphology import remove_small_objects
from skimage.morphology import skeletonize


import time 

Using TensorFlow backend.


In [2]:
basedir = '/run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a'

basedir_StardistWater_StardistMask = '/run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/StarWat_StarMask/'
basedir_StardistWater_StardistMask_RTGPipeline = '/run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/CycE04/StardistWater_StardistMask_RTGPipeline/'
basedir_StardistWater_UnetMask = '/run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/StarWat_UnetMask/'
basedir_StardistMask = '/run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/StarMask/'
basedir_UnetMask = '/run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/UnetMask/'

Model_Dir = '/media/sancere/Newton_Volume_1/CurieDeepLearningModels/DrosophilaSegmentation'    

In [3]:
StardistModelName = 'DrosophilaSegmentationSmartSeedsMatureMacroElongatedBoris'
UnetMaskModelName = 'DrosophilaMaskSegmentationCARE'

modelstar = StarDist2D(config = None, name = StardistModelName, basedir = Model_Dir)
modelmask = CARE(config = None, name = UnetMaskModelName, basedir = Model_Dir)






Loading network weights from 'weights_best.h5'.






Couldn't load thresholds from 'thresholds.json', using default values. (Call 'optimize_thresholds' to change that.)
Using default values: prob_thresh=0.5, nms_thresh=0.4.
Loading network weights from 'weights_best.h5'.


In [4]:
# Path(basedir_StardistWater_StardistMask).mkdir(exist_ok = True)
Path(basedir_StardistWater_UnetMask).mkdir(exist_ok = True)
# Path(basedir_StardistMask).mkdir(exist_ok = True)
# Path(basedir_UnetMask).mkdir(exist_ok = True)

Raw_path = os.path.join(basedir, '*tif') #tif or TIF be careful

axis_norm = (0,1)   # normalize channels independently
axes = 'XY'

filesRaw = glob.glob(Raw_path)
# filesRaw.sort

In [8]:
for fname in filesRaw:
 
    if os.path.exists((basedir_StardistWater_UnetMask + 'StarWat_UnetMask_' + os.path.basename(os.path.splitext(fname)[0]))) == False :
#     if os.path.exists(basedir_StardistWater_StardistMask + 'StarWat_StarMask_' + os.path.basename(os.path.splitext(fname)[0])) == False \
#     or os.path.exists((basedir_StardistMask + 'StarMask_' + os.path.basename(os.path.splitext(fname)[0]))) == False \
#     or os.path.exists((basedir_UnetMask + 'UnetMask_' + os.path.basename(os.path.splitext(fname)[0]))) == False :
                     
            # Name = os.path.basename(os.path.splitext(fname)[0])
            x = imread(fname)
            y = normalizeFloat(x,1,99.8, axis = axis_norm)
            originalX = x.shape[0]
            originalY = x.shape[1]
        
            #zero pad aand 8bit conversion after normalization
            y = zero_pad(y, 32, 32)
            print('Processing file', fname)              
           
            #Stardist, label image, details, probability map, distance map 
            segmentationImage, details = modelstar.predict_instances(y)
            prob, dist = modelstar.predict(y)
            grid = modelstar.config.grid
            prob = cv2.resize(prob,  dsize=(prob.shape[1] * grid[1],prob.shape[0] * grid[0]))
            
            
            #Seeds from Stardist + Whatershed, segmentation on probability map 
            Watershed, markers = SeedStarDistWatershed(prob,details['points'],modelstar.config.grid) 
                                                                   
            StardistWater_UnetMask
            #Convert Integer image to binary
            Binary_Watershed = Integer_to_border(Watershed[:originalX, :originalY])
                       
                                                       
            #Generation of masks               
#             StardistMask = Prob_to_Binary(prob[:originalX, :originalY], segmentationImage[:originalX, :originalY])
              
            HomeMadeMask = cv2.imread(fname.replace('.tif','_Mask.png'))
            HomeMadeMask = HomeMadeMask[:,:,0]
            UnetMask = HomeMadeMask
            
#             UnetMask = modelmask.predict(x, axes, n_tiles = (1, 2))
#             UnetMask = UnetMask.astype('uint8')
#             thresh = threshold_otsu(UnetMask)
#             UnetMask = UnetMask > thresh*0.1
#             UnetMask = remove_small_objects(UnetMask, 5000)
#             UnetMask = UnetMask.astype('uint8')
            
                                                                   
            #Create StardistWater_StardistMask and StardistWater_UnetMask with logical and operation            
#             StardistWater_StardistMask = np.logical_and(StardistMask, Binary_Watershed)
            StardistWater_UnetMask = np.logical_and(UnetMask, Binary_Watershed)
           
        
            #Skeletonization and deletion of non closed cells (only for StardistWater_StardistMask here)
            z = StardistWater_UnetMask 
            z = skeletonize(z) 
            z = np.float32(z)
            z2 = z.copy()
            mask = np.zeros((np.array(z.shape)+2), np.uint8)
            cv2.floodFill(z, mask, (0,0), (255))
            z = cv2.erode(z, np.ones((3,3)))
            z = cv2.bitwise_not(z)
            z = cv2.bitwise_and(z,z2)
            StardistWater_UnetMask = z 
    
            #Save different method segmentations
#             save_8bit_tiff_imagej_compatible((basedir_StardistWater_StardistMask + 'StarWat_StarMask_' + os.path.basename(os.path.splitext(fname)[0])) ,  StardistWater_StardistMask, axes)
            save_8bit_tiff_imagej_compatible((basedir_StardistWater_UnetMask + 'StarWat_UnetMask_' + os.path.basename(os.path.splitext(fname)[0]) ) , StardistWater_UnetMask, axes)
#             save_8bit_tiff_imagej_compatible((basedir_StardistMask + 'StarMask_' + os.path.basename(os.path.splitext(fname)[0])) , StardistMask, axes)
#             save_8bit_tiff_imagej_compatible((basedir_UnetMask + 'UnetMask_' + os.path.basename(os.path.splitext(fname)[0]) ) , UnetMask, axes)

Processing file /run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/vi2a_0186.tif
Processing file /run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/vi2a_0009.tif
Processing file /run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/vi2a_0157.tif
Processing file /run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/vi2a_0212.tif
Processing file /run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/vi2a_0100.tif
Processing file /run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/vi2a_0034.tif
Processing file /run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/vi2a_0191.tif

In [11]:
from PIL import Image 
import png


basedir_StardistWater_UnetMask_RTGPipeline = '/run/user/1001/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/vi2a/StardistWater_UnetMask_RTGPipeline/'
Path(basedir_StardistWater_UnetMask_RTGPipeline).mkdir(exist_ok = True)

Raw_path_2 = os.path.join(basedir_StardistWater_UnetMask, '*') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw_2 = glob.glob(Raw_path_2)


name_of_file = "Lucas Reminder Format"
completeName = os.path.join(basedir_StardistWater_UnetMask_RTGPipeline, name_of_file + ".txt")         
file = open(completeName, "w")
file.write("If Matlab Pipeline error appears it might be the format '8bit-color'. Can't be generated by python. Run the Pipeline from Cleaning Filter. Email Stephane for LOCALISATION in Pipeline")
file.close()


    
for fname in filesRaw_2:
    z = imread(fname)
    z = 255 * z
    z = cv2.bitwise_not(z)
    name = str(os.path.basename(os.path.splitext(fname)[0])).rstrip(".tif")
    png.from_array(z, mode="L").save(basedir_StardistWater_UnetMask_RTGPipeline + name.replace("StarWat_UnetMask",'seg_') + '.png') 
